In [37]:
import requests
import pandas as pd
import os
import requests
from mainnet_launch.constants import ChainData, ETH_CHAIN

API_URL = "https://v2-config.tokemaklabs.com/api/systems"


def fetch_systems() -> list[dict]:
    """Fetch the raw systems JSON (raises on HTTP errors)."""
    resp = requests.get(API_URL)
    resp.raise_for_status()
    return resp.json()


def build_deployers_df(systems: list[dict]) -> pd.DataFrame:
    """One row per deployer (chainId, deployer)."""
    rows = []
    for sys in systems:
        cid = sys["chainId"]  # KeyError if missing
        for deployer in sys["deployers"]:  # KeyError if missing
            rows.append({"chainId": cid, "deployer": deployer})
    return pd.DataFrame(rows)


def build_keepers_df(systems: list[dict]) -> pd.DataFrame:
    """One row per Chainlink keeper (chainId + keeper fields)."""
    rows = []
    for sys in systems:
        cid = sys["chainId"]
        for keeper in sys["chainlinkKeepers"]:  # KeyError if missing
            keeper_row = {
                "chainId": cid,
                "name": keeper["name"],
                "id": keeper["id"],
                "url": keeper["url"],
                "deprecated": keeper["deprecated"],
            }
            rows.append(keeper_row)
    return pd.DataFrame(rows)


def build_service_accounts_df(systems: list[dict]) -> pd.DataFrame:
    """One row per service account (chainId + account fields)."""
    rows = []
    for sys in systems:
        cid = sys["chainId"]
        for acct in sys["serviceAccounts"]:  # KeyError if missing
            acct_row = {
                "chainId": cid,
                "name": acct["name"],
                "address": acct["address"],
                "type": acct["type"],
            }
            rows.append(acct_row)
    return pd.DataFrame(rows)


def fetch_systems_df():
    systems = fetch_systems()
    deployers_df = build_deployers_df(systems)
    chainlink_keepers_df = build_keepers_df(systems)
    service_accounts_df = build_service_accounts_df(systems)
    return deployers_df, chainlink_keepers_df, service_accounts_df


deployers_df, chainlink_keepers_df, service_accounts_df = fetch_systems_df()
deployers_df

# for simplicity this only looks at gas costs on mainnet
# sonic, and base are near 0 anyway

,chainId,deployer
0,146,0x5552e8b0BD553C299Fbf9731c92dD6C3B5AB3950
1,8453,0xba5e4322fe1c3c23Cb74304fE9537bF54a6B781E
2,1,0x123cC4AFA59160C6328C0152cf333343F510e5A3


In [ ]:
from mainnet_launch.constants import *


def get_outgoing_transactions(
    chain: ChainData, address: str, from_block: int, to_block: int, page_key: str | None
) -> tuple[pd.DataFrame, str | None]:
    """
    Fetches all transfers sent *from* `address` between `from_block` and `to_block` on chain
    by calling Alchemy's JSON-RPC API via HTTP.

    page key, if present, can be used to paginate through results.
    If no page key is present, all results have been returned.
    """
    categories = ["external", "internal", "erc20", "erc721", "erc1155", "specialnft"]

    params = [
        {
            "fromBlock": hex(from_block),
            "toBlock": hex(to_block),
            "fromAddress": chain.client.toChecksumAddress(address),
            "category": categories,  # not totally certain here
        }
    ]
    if page_key is not None:
        params[0]["pageKey"] = page_key

    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "alchemy_getAssetTransfers",
        "params": params,
    }

    resp = requests.post(chain.client.provider.endpoint_uri, json=payload)
    resp.raise_for_status()
    data = resp.json()
    result = data["result"]
    page_key: str | None = result.get("pageKey")

    return result["transfers"], page_key


def get_all_transactions_sent_by_eoa_address(
    chain: ChainData, EOA_address: str, from_block: int, to_block: int
) -> pd.DataFrame:
    """Use pagination, to get all transactions sent by an EOA address on a given chain."""
    records = []
    print(f"{from_block=}, {to_block=}, {EOA_address=}")
    page_key = None
    while True:
        transfers, page_key = get_outgoing_transactions(
            chain=chain, address=EOA_address, from_block=from_block, to_block=to_block, page_key=page_key
        )
        records.append(transfers)

        if page_key is None:
            # if there is no page key, we have all results
            break

    return transfers


mainnet_deployer = "0x123cC4AFA59160C6328C0152cf333343F510e5A3"
local_wallet = "0x76fb6D38F28C44a13380220Df21363bD7Af45ee1"

tx_df = get_all_transactions_sent_by_eoa_address(
    ETH_CHAIN,
    local_wallet,
    from_block=ETH_CHAIN.block_autopool_first_deployed,
    to_block=ETH_CHAIN.client.eth.block_number,
)

tx_df

from_block=20722908, to_block=22884361, EOA_address='0x76fb6D38F28C44a13380220Df21363bD7Af45ee1'


[{'blockNum': '0x1465ef3',
  'uniqueId': '0xcbe7e54790ea32adc821bbb3feac6bf06cea3af2c9d771bdcd5653f2d653c2bf:external',
  'hash': '0xcbe7e54790ea32adc821bbb3feac6bf06cea3af2c9d771bdcd5653f2d653c2bf',
  'from': '0x76fb6d38f28c44a13380220df21363bd7af45ee1',
  'to': '0xb8697f7a140a3e4087fa640f978c68b3ecdac688',
  'value': 0.01,
  'erc721TokenId': None,
  'erc1155Metadata': None,
  'tokenId': None,
  'asset': 'ETH',
  'category': 'external',
  'rawContract': {'value': '0x2386f26fc10000',
   'address': None,
   'decimal': '0x12'}}]

In [ ]:
import os
import requests
import pandas as pd
from mainnet_launch.constants import ChainData

# You’ll need to have your Etherscan key in env var ETHERSCAN_API_KEY
ETHERSCAN_API_URL = "https://api.etherscan.io/api"
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")


def get_outgoing_transactions(
    chain: ChainData,
    address: str,
    from_block: int,
    to_block: int,
    page: int = 1,
    offset: int = 1000,
) -> tuple[list[dict], bool]:
    """
    Fetches up to `offset` internal txns sent *from* `address` between
    `from_block` and `to_block` on chain via Etherscan.
    Returns (tx_list, has_more) where `has_more` is True if we got a full page.
    """
    params = {
        "module": "account",
        "action": "txlist",
        "chainid": chain.chain_id,  # e.g. 1 for mainnet
        "address": str.lower(address),
        "startblock": from_block,
        "endblock": to_block,
        "page": page,
        "offset": offset,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY,
    }
    print(params)

    resp = requests.get(ETHERSCAN_API_URL, params=params)
    resp.raise_for_status()
    payload = resp.json()

    # Etherscan returns status "1" for success with non-empty, "0" for empty or error
    if payload.get("status") != "1":
        return [], False

    txs = payload.get("result", [])
    # if we received exactly `offset` entries, there *may* be more on next page
    has_more = len(txs) == offset
    return txs, has_more


def get_all_transactions_sent_by_eoa_address(
    chain: ChainData,
    EOA_address: str,
    from_block: int,
    to_block: int,
    page_size: int = 1000,
) -> pd.DataFrame:
    """Use pagination to get *all* internal txns sent by `EOA_address`."""
    all_txs: list[dict] = []
    page = 1

    while True:
        txs, has_more = get_outgoing_transactions(
            chain=chain,
            address=EOA_address,
            from_block=from_block,
            to_block=to_block,
            page=page,
            offset=page_size,
        )
        if not txs:
            break

        all_txs.extend(txs)
        if not has_more:
            break

        page += 1

    # convert list of dicts into DataFrame
    return pd.DataFrame(all_txs)


# — example usage —
mainnet_deployer = "0x123cC4AFA59160C6328C0152cf333343F510e5A3"
# a = get_outgoing_transactions(ETH_CHAIN, local_wallet, ETH_CHAIN.block_autopool_first_deployed, ETH_CHAIN.client.eth.block_number)

tx_df = get_all_transactions_sent_by_eoa_address(
    ETH_CHAIN,
    local_wallet,
    from_block=ETH_CHAIN.block_autopool_first_deployed,
    to_block=ETH_CHAIN.client.eth.block_number,
)

# # print(tx_df)

{'module': 'account', 'action': 'txlist', 'chainid': 1, 'address': '0x76fb6d38f28c44a13380220df21363bd7af45ee1', 'startblock': 20722908, 'endblock': 22884420, 'page': 1, 'offset': 1000, 'sort': 'asc', 'apikey': 'U6GVUZZ4DXG6G6FAGCIP7UAM72THP6XDRN'}


In [60]:
tx_df

,blockNumber,blockHash,timeStamp,hash,nonce,transactionIndex,from,to,value,gas,gasPrice,input,methodId,functionName,contractAddress,cumulativeGasUsed,txreceipt_status,gasUsed,confirmations,isError
0,21389043,0xf7f2acab8044e11ca8ca061a6c138dcc6c615dbae8e0...,1734038039,0xcbe7e54790ea32adc821bbb3feac6bf06cea3af2c9d7...,82,186,0x76fb6d38f28c44a13380220df21363bd7af45ee1,0xb8697f7a140a3e4087fa640f978c68b3ecdac688,10000000000000000,21000,18584596666,0x,0x,,,16843561,1,21000,1495378,0


In [35]:
deployers_df

NameError: name 'deployers_df' is not defined

In [4]:
chainlink_keepers_df

,chainId,name,id,url,deprecated
0,8453,Incentive Pricing,8188722995180760360617733804019303233643645174...,https://automation.chain.link/base/81887229951...,False
1,8453,Calculators,7666724683051043009392549723240027209945628421...,https://automation.chain.link/base/76667246830...,False
2,1,Incentive Pricing,8491081058992380159853603150782794192373563166...,https://automation.chain.link/mainnet/84910810...,False
3,1,Eth Per Token Sender,2774403708484311544165440706031341871504925629...,https://automation.chain.link/mainnet/27744037...,False
4,1,Calculators,1344461886831441856282597505993515040672606510...,https://automation.chain.link/mainnet/13444618...,True
5,1,Calculators,1131296732650549075674204606512778729971626443...,https://automation.chain.link/mainnet/11312967...,True
6,1,Calculators,9344370690633218040753518430381561629034314154...,https://automation.chain.link/mainnet/93443706...,False


In [3]:
deployers_df

,chainId,deployer
0,146,0x5552e8b0BD553C299Fbf9731c92dD6C3B5AB3950
1,8453,0xba5e4322fe1c3c23Cb74304fE9537bF54a6B781E
2,1,0x123cC4AFA59160C6328C0152cf333343F510e5A3


In [ ]:



address = "0xb5d85CBf7cB3EE0D56b3bB207D5Fc4B82f43F511"
.keys()


dict_keys(['transfers', 'pageKey'])

In [32]:
address = "0x76fb6d38f28c44a13380220df21363bd7af45ee1"
get_outgoing_transactions(address, ETH_CHAIN)["result"].keys()

dict_keys(['transfers'])

In [ ]:
tx_list

dict_keys(['transfers', 'pageKey'])

In [ ]:
break

In [25]:
tx_list.keys()

dict_keys(['jsonrpc', 'id', 'result'])

In [20]:
len(tx_list["result"]["transfers"])

1000

In [11]:
url

'https://eth-mainnet.g.alchemy.com/v2/7dyyKalTzbS0cySjdpnkr'

In [5]:
service_accounts_df

,chainId,name,address,type
0,146,Custom Oracle Executor,0x8d096E2b5Fe29B914dB11F3ecE13b6E7F72fA2a2,pricing
1,146,Solver Exec (autoS),0x84C75b7C35c121849DA06DF6946A6C9179B5edE2,solver
2,146,Solver Exec (sonicUSD),0xE71beF438317dC39c2c946809831A87EcE048857,solver
3,146,Liquidator,0x28Fa1720cB4e311695CFaA726C2A34b028af0b97,rewards
4,146,Swap Route Watcher,0x0a3A597058fc2A344cCA32653ca082C775d8925d,swaps
5,146,Debt Reporting,0x6dE76a011706315bAf1eAad329B961B1B30D33C9,debtReporting
6,8453,Custom Oracle Executor,0x8CD98E74A5ef4020bcd6D0397EFe42C34Afa5f23,pricing
7,8453,Debt Reporting,0xb2A71EF7649B95d7d5A3b8084e619C381Fa79413,debtReporting
8,8453,Swap Route Watcher,0xf772Fcd65aCAE9814e6E4b228e554c1054aE2765,swaps
9,8453,Incentive Harvester,0xcA7d8b1F4398D11b302853d7fFd5D2b65073dE11,stats
